In [11]:
%pylab inline
import matplotlib.pyplot as plt 
import numpy as np
import cv2
np.seterr(all='raise')

Populating the interactive namespace from numpy and matplotlib


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [12]:
def nd(l):
    return np.array(l,dtype=float)
def Vis(U):
# Visualizing A Cluster
    img=U/U.max()
    img=(img*255).astype(int)
    plt.imshow(img,cmap='gray')
    plt.show()
    return img
def VisCrisp(U):
    img=(U==numpy.amax(U,axis=(0)))*255
    Map=[]
    for Map in img:
        Map.append(Vis(Map))
    return Map

In [13]:
def normalizeImage(img):
    minPx,maxPx=img.min(),img.max()
    NormalizedImg = (img-minPx)/(maxPx-minPx)
    return NormalizedImg

In [42]:
def sugenoNeg(mu,b=0.7):
    return (1-mu)/(1+b*mu)

def randomUMatGen(clusters,img_shape):
    #U is the Uij matrix
    U = np.random.rand(clusters,img_shape[0],img_shape[1])
    U = U / U.sum(axis=0)
    return U

def UpCen(U,x):
    return ((U*x).sum(axis=(1,2))) / (U.sum(axis=(1,2)))

def getCentroids(X,U,m=2):
    U=1-sugenoNeg(U)
    muS= UpCen(np.power(U,m),X[0])
    nuS= UpCen(np.power(U,m),X[1])
    piS= UpCen(np.power(U,m),X[2])
    V = np.array([muS,nuS,piS])
    return V

def d2(X,Y):
    return (np.power(X-Y,2)).sum(axis=0)

def updateMatU(X,V,m):
    delta = float("0.1e-7")
    D2=d2(X[:,np.newaxis,:,:],V[:,:,np.newaxis,np.newaxis])+delta
    D2=np.power(D2,-1/(m-1))
    D2=D2/D2.sum(axis=0)
    return D2

In [30]:
def fcm(mu,m=2,c=4,e=0.0001,maxIter=400):
    nu=sugenoNeg(mu)
    X=np.array([mu,nu,1-mu-nu])
    U=randomUMatGen(c,mu.shape)
    V=getCentroids(X,U)

    ind=1
    U_prev=zeros_like(U)
    
    while ind<maxIter:
        if (np.absolute(U-U_prev)).max() < e:
            break
        U_prev=U
        U=updateMatU(X,V,m)
        V=getCentroids(X,U)
        ind+=1
    print(ind)
    return U[ind%2],V    

In [16]:
def printImg(img):
    plt.imshow(img,cmap='gray')
    plt.show()
    
def GetImg(FileName):
    import os
    cwd = os.getcwd()
    img = cv2.imread(cwd+'\\Img\\'+FileName, cv2.IMREAD_GRAYSCALE)
    return img

def GetMap(U):
    Map=(U==numpy.amax(U,axis=(0))).astype(int)
    return Map

def GetGroundTurthMaps(FileName='NewImg2.tiff'):
    import os
    cwd = os.getcwd()
    img1 = GetImg(FileName)
    uniq=np.unique(img1)
    uniq.sort()
    Map=[]
    for i in range(uniq.size):
        Map.append(img1==uniq[i])
    return nd(Map),uniq,img1

def GetResMaps(res,getGrndTruth=0):
    img=GetImg('NewImg2.tiff')
    Maps=GetMap(res[0])
    Maps=nd(Maps)
    Avg=nd([(img*Map).sum()/Map.sum() for Map in Maps])
    Ord=numpy.argsort(Avg)
    Maps=Maps[Ord]
    #Maps=Maps*Inten[:,np.newaxis,np.newaxis] #To get the colored resultant
    return Maps

In [38]:
def DiceScore(G,R):
    intersectionPx=2*(G*R).sum(axis=(1,2))
    return intersectionPx/(G+R).sum(axis=(1,2))
def AvgSegAcc(G,R):
    intersectionPx=(G*R).sum()
    return intersectionPx/G.sum()
def Scores(G,R):
    return DiceScore(G,R),AvgSegAcc(G,R)

def GetScores(FileName,G,printResMap=1):
    res=fcm(GetImg(FileName),maxIter=1000,e=0.00001)
    R=GetResMaps(res)
    if printResMap:
        for seg in R:
            printImg(seg)
    return Scores(G,R)

def GetAllImgScores(printResMap=0):
    import os
    path = os.getcwd()+'\\Img'
    Files=os.listdir(path)
    G,a,b=GetGroundTurthMaps()
    FilesScr={}
    for file in Files:
        FilesScr[file]=GetScores(file,G,printResMap)
        #print(FilesScr[file])
    return FilesScr

In [34]:
GetAllImgScores()

14
21
63
36
37


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.74034283, 0.99652405, 0.81549464, 0.99248061]),
  0.9448517126148706),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [39]:
GetAllImgScores()

12
34
26
42
46


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.7682551 , 1.        , 0.85588704, 0.99980823]),
  0.9589129072681705),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [40]:
GetAllImgScores()

15
32
27
42
63


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.7682551 , 1.        , 0.85588704, 0.99980823]),
  0.9589129072681705),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [43]:
GetAllImgScores()

14
36
29
44
45


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.7682551 , 1.        , 0.85588704, 0.99980823]),
  0.9589129072681705),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [44]:
GetAllImgScores()

13
33
31
32
50


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [45]:
GetAllImgScores()

11
33
28
41
44


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.7682551 , 1.        , 0.85588704, 0.99980823]),
  0.9589129072681705),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [46]:
GetAllImgScores()

12
33
29
31
46


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [47]:
GetAllImgScores()

13
25
20
40
52


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [48]:
GetAllImgScores()

12
24
37
65
46


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.70318396, 0.77650442, 0.8547523 , 0.97120069]),
  0.9073673767752715),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}

In [49]:
GetAllImgScores()

12
29
22
50
60


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([0.54681491, 0.75094977, 0.99856841, 0.99980823]),
  0.9319705513784461),
 'NewImg2_0_002.tiff': (array([0.58133661, 0.75634944, 0.96208531, 0.99367554]),
  0.9264306599832915),
 'NewImg2_0_003.tiff': (array([0.59756386, 0.75673012, 0.87332725, 0.9750173 ]),
  0.9029291979949875),
 'NewImg2_0_004.tiff': (array([0.61448715, 0.75969023, 0.728872  , 0.93157434]),
  0.8509346282372598)}